# More Practice With SQL Queries - Lab

## Introduction

In this lesson, we'll run through some practice questions to refresh your knowledge of SQL Queries!

## Objectives

You will be able to:

- Use `GROUP BY` statements in SQL to apply aggregate functions like: `COUNT`, `MAX`, `MIN`, and `SUM`
- Decide and perform whichever type of join is best for retrieving desired data
- Use the `HAVING` clause to compare different aggregates
- Write subqueries to decompose complex queries

## Getting Started

As in previous labs, we'll make use of the `sqlite3` library as well as Pandas. By combining them, we'll be able to write queries as Python strings, and make sure that the results are always returned as a Pandas DataFrame. 

We'll start by loading both libraries and connecting to the database we'll be using for this lab, `data.sqlite`. You may remember this database from a previous lab. As a refresher, here's the ERD diagram for this database: 

<img src='images/Database-Schema.png'>

In the cell below:

* Import the necessary libraries `pandas` and `sqlite3` 
* Establish a connection to the database `data.sqlite` 
* Get the `cursor` from the connection and store it in the variable `c` 

In [1]:
import pandas as pd
import sqlite3
conn = sqlite3.connect('data.sqlite')
c = conn.cursor()

## Basic Queries

Now, let's review basic SQL queries. In the cell below:

Write a query that gets the first name, last name, phone number, address, and credit limit for all customers in California with a credit limit greater than 25000.00. 

In [ ]:
c.execute(""" """)
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df

In [8]:
c.execute("""SELECT contactFirstName, contactLastName, phone, addressLine1, creditLimit
            FROM customers
            WHERE creditLimit > 25000 AND state == "CA";
""")
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df

,contactFirstName,contactLastName,phone,addressLine1,creditLimit
0,Susan,Nelson,4155551450,5677 Strong St.,210500
1,Julie,Murphy,6505555787,5557 North Pendale Street,64600
2,Juri,Hashimoto,6505556809,9408 Furth Circle,84600
3,Julie,Young,6265557265,78934 Hillside Dr.,90700
4,Valarie,Thompson,7605558146,361 Furth Circle,105000
5,Julie,Brown,6505551386,7734 Strong St.,105000
6,Brian,Chandler,2155554369,6047 Douglas Av.,57700
7,Sue,Frick,4085553659,3086 Ingle Ln.,77600
8,Steve,Thompson,3105553722,3675 Furth Circle,55400
9,Sue,Taylor,4155554312,2793 Furth Circle,60300


#### Expected Output

<img src='images/expected-output-1.png'>

## Aggregate Functions and GROUP BY

Next, write a query that gets the average credit limit per state.

In [12]:
c.execute(""" 
                SELECT AVG(creditLimit) as average_credit_limit, state
                FROM customers
                GROUP BY state

""")
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df

,average_credit_limit,state
0,61839.726027,
1,89950.000000,BC
2,83854.545455,CA
3,57350.000000,CT
4,0.000000,Co. Cork
5,93900.000000,Isle of Wight
6,70755.555556,MA
7,114200.000000,NH
8,43000.000000,NJ
9,100550.000000,NSW


#### Expected Output

<img src='images/expected-output-2.png'>

## JOINs

Now, write a query that uses JOIN statements to get the customer name, customer number, order number, status, and quantity ordered. Print only the head of this DataFrame. 

In [14]:
c.execute("""
            SELECT customerName, customerNumber, orderNumber, status, quantityOrdered
            FROM customers
            JOIN orders 
            USING(customerNumber)
            JOIN orderDetails
            USING(orderNumber)
            
""")
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()

,customerName,customerNumber,orderNumber,status,quantityOrdered
0,Atelier graphique,103,10123,Shipped,26
1,Atelier graphique,103,10123,Shipped,34
2,Atelier graphique,103,10123,Shipped,46
3,Atelier graphique,103,10123,Shipped,50
4,Atelier graphique,103,10298,Shipped,32


#### Expected Output

<img src='images/joins.png' width='500' height='600'> 

## HAVING and ORDER BY

Now, return the customerName, customerNumber, productName, productCode, and total number ordered for any product a customer has bought 10 or more of cumulatively. Sort the rows in descending order by the quantity ordered. 

**_Hint_**: For this one, you'll need to make use of HAVING, GROUP BY, and ORDER BY -- make sure you get the order of them correct!

In [28]:
c.execute("""SELECT c.customerName, c.customerNumber, p.productName, p.productCode, SUM(od.quantityOrdered) as num
            FROM Customers c JOIN Orders o
                    ON c.customerNumber = o.customerNumber
                    JOIN Orderdetails od
                    USING(orderNumber)
                    JOIN Products p
                    USING(productCode)
            GROUP BY c.customerNumber, productCode
            HAVING num > 10
            ORDER BY num DESC;

""")
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()

,customerName,customerNumber,productName,productCode,num
0,Euro+ Shopping Channel,141,1992 Ferrari 360 Spider red,S18_3232,308
1,Euro+ Shopping Channel,141,1958 Chevy Corvette Limited Edition,S24_2840,245
2,Euro+ Shopping Channel,141,1970 Dodge Coronet,S24_1444,197
3,Euro+ Shopping Channel,141,1957 Chevy Pickup,S12_4473,183
4,Euro+ Shopping Channel,141,2002 Chevy Corvette,S24_3432,174


#### Expected Output

<img src='images/having_order.png'>

## Subqueries

Finally, get the first name, last name, employee number, and office code for employees from offices with less than 5 employees. Print the first five rows of this DataFrame. 

In [39]:
c.execute("""SELECT lastName, firstName, employeeNumber, officeCode
            FROM employees
            WHERE officeCode IN (SELECT officeCode
                                FROM offices
                                JOIN employees
                                USING (officeCode)
                                GROUP BY officeCode
                                HAVING COUNT(employeeNumber) < 5)
            GROUP BY officeCode;

""")
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df

,lastName,firstName,employeeNumber,officeCode
0,Firrelli,Julie,1188,2
1,Tseng,Foon Yue,1286,3
2,Nishi,Mami,1621,5
3,Patterson,William,1088,6
4,Bott,Larry,1501,7


In [37]:
c.execute("""SELECT *
            FROM employees
      ;

""")
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df

,employeeNumber,lastName,firstName,extension,email,officeCode,reportsTo,jobTitle
0,1002,Murphy,Diane,x5800,dmurphy@classicmodelcars.com,1,,President
1,1056,Patterson,Mary,x4611,mpatterso@classicmodelcars.com,1,1002,VP Sales
2,1076,Firrelli,Jeff,x9273,jfirrelli@classicmodelcars.com,1,1002,VP Marketing
3,1088,Patterson,William,x4871,wpatterson@classicmodelcars.com,6,1056,Sales Manager (APAC)
4,1102,Bondur,Gerard,x5408,gbondur@classicmodelcars.com,4,1056,Sale Manager (EMEA)
5,1143,Bow,Anthony,x5428,abow@classicmodelcars.com,1,1056,Sales Manager (NA)
6,1165,Jennings,Leslie,x3291,ljennings@classicmodelcars.com,1,1143,Sales Rep
7,1166,Thompson,Leslie,x4065,lthompson@classicmodelcars.com,1,1143,Sales Rep
8,1188,Firrelli,Julie,x2173,jfirrelli@classicmodelcars.com,2,1143,Sales Rep
9,1216,Patterson,Steve,x4334,spatterson@classicmodelcars.com,2,1143,Sales Rep


#### Expected Output

<img src='images/expected-output-5.png'>

# Summary

In this lesson, we reviewed all the major concepts and keywords associated with SQL queries!